In [1]:
from stonks.data.data_reader import MktDataReader
from stonks.models.market_models import MarketModels
from stonks.models.asset_allocation import AssetAllocation


from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions 

In [2]:
tickers = [
    'AAPL', # Apple -> tech
    'TSLA', # Tesla -> tech
    'GOOGL', # Google -> tech
    'AMZN', # Amazon -> consumer/tech
    'MA', # MasterCard -> finance
    'V', # Visa -> finance
    'MELI', # MercadoLibre -> consumer
    'NVDA', # Nvidia -> chips
    'ASML', # ASML -> chips
    'FB' # facebook -> social media
]

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-04-05'

dr = MktDataReader(start_date, end_date, 'yahoo', tickers)

[*********************100%***********************]  10 of 10 completed


In [3]:
# impute missing data
dr.impute_missing_data()
# fetch market caps
dr.fetch_market_caps()

In [4]:
## Black-Litterman

In [5]:
# declare our views of the market
viewdict = {
    "GOOGL": 0.30,
    "FB": 0.30,
    "ASML": 0.30,
    "NVDA": 0.30,
}

# how confident we are on our views
confidences = [
    0.8,
    0.8,
    0.6,
    0.6,
]

# initiate the stonks MarketModels obj
mmds = MarketModels(
    dr.df_stocks_bkfilled,  # stocks historical data from the MktDataReader class
    model = "bl",  # name of the model to get the estimated returns + covar matrices
    views_dict = viewdict,  # own views on expectations of stocks
    confidences = confidences,  # how sure are you about your views of the market
    mcaps = dr.mcaps  # market caps from the MktDataReader class
) 

[*********************100%***********************]  1 of 1 completed


In [6]:
# which model we used + the trained model obj
mmds.model

{'name': 'Black-litterman',
 'model': <pypfopt.black_litterman.BlackLittermanModel at 0x7f851891a8d0>}

In [7]:
print(type(mmds.ret_bl))
print(type(mmds.S_bl))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
# asset allocation

In [16]:
# using the computed black-litterman model
al = AssetAllocation(mmds.ret_bl, mmds.S_bl, "markowitz-regularized")
al.portfolio_perf

Expected annual return: 28.2%
Annual volatility: 25.8%
Sharpe Ratio: 1.02


(0.28230971005424477, 0.2584102165904742, 1.0150903223379533)

In [15]:
# portfolio weights
al.weights

OrderedDict([('AAPL', 0.10436),
             ('AMZN', 0.11536),
             ('ASML', 0.09508),
             ('FB', 0.12081),
             ('GOOGL', 0.11316),
             ('MA', 0.0856),
             ('MELI', 0.10017),
             ('NVDA', 0.10322),
             ('TSLA', 0.08598),
             ('V', 0.07626)])